<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/data_utils/munich/translate_listings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Translation of airbnb dataset into english

In [1]:
from google.colab import drive
import pandas as pd
import torch
drive.mount('/content/drive')
!git clone https://github.com/dernameistegal/airbnb_price.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'airbnb_price' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append("/content/airbnb_price/custom_functions")
import general_utils as ut
device = ut.get_device()

cuda available: False ; cudnn available: True ; num devices: 0
Using device cpu


## Preliminary works
- add comments_en column with values None
- add lang column with values None
- replace na values in comments colum with "no text"

In [16]:
%%capture
!wget http://data.insideairbnb.com/germany/bv/munich/2021-12-24/data/listings.csv.gz

In [4]:
listings = pd.read_csv("listings.csv.gz")
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"

In [23]:
keep_ids, _, _ = ut.train_val_test_split(listings.index, train_size=0.20)  
listings = listings.loc[keep_ids]
listings.to_pickle(path)

In [5]:
listings = pd.read_pickle(path)

In [6]:
listings.index

Int64Index([ 987, 2549,  193, 3057, 3246, 1394, 2637,  768, 4697, 3090,
            ...
            1685,  130, 2919, 3171, 3444, 4426,  466, 3092, 3772,  860],
           dtype='int64', length=999)

In [26]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]
# remember to restart runtime

In [8]:
# todo
listings.description
listings.host_about
listings.name

987            3 rooms near underground station in Munich
2549          Wohnung mitten im Trendviertel Gärtnerplatz
193                    Central apartment near Oktoberfest
3057        Zimmer nähe Oktoberfest (5 Min.) an Bedienung
3246    Cosy Flat near to Oktoberfest with Balcony and...
                              ...                        
4426            Gemütliche Wohnung in schönster Isar-Lage
466     Garden House Apartment -4 Bedrooms (6-15 perso...
3092            Gemütliche 1,5 Zimmer Wohnung Zentrumsnah
3772                Luxury Studio in the centre of Munich
860        Separate entrance and bathroom, quiet location
Name: name, Length: 999, dtype: object

In [9]:
listings[["description_en"]] = listings[["description"]]
listings["lang"] = listings["description"]
listings[["description_en", "lang"]] = None

In [10]:
listings[["description", "host_about", "name"]] = listings[["description", "host_about", "name"]].fillna("no text")

In [11]:
from torch.utils.data import DataLoader, Dataset

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

In [12]:
dataset = Translatedata(listings, "description")
dataloader = DataLoader(dataset, batch_size=200)

In [13]:
# remove text garbage	
listings["description"] = listings["description"].str.replace("<.*?>", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [14]:
listings["description"].iloc[0]

'Das ist eine 3 Raum Wohnung, Küche mit Waschmaschine, Kaffeemaschine, Wasserkocher, Toster und allem, was auf den Bildern zu sehen ist, WLAN und TV in jedem Zimmer. Bis zur U-Bahn Forstenrieder Allee sind 4 Min. zu Fuß, dann 10 Min. bis zum Marienplatz. Neben der U-Bahn Station befinden sich REWE, Bowling, Ca\xadfé, dm-drogerie markt.Die Lage ist absolut ruhig. Bettwäsche, TücherМы говорим по русски!'

In [17]:
from txtai.pipeline import Translation
translate = Translation(batch=200)
translate(listings["description"].iloc[0])

Downloading:   0%|          | 0.00/916k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

'This is a 3 room apartment, kitchen with washing machine, coffee maker, kettle, toster and everything that can be seen in the pictures, Wi-Fi and TV in each room. Up to the underground Forstenrieder avenue are 4 min. walk, then 10 min. to Marienplatz. Next to the underground station are REWE, bowling, ca-fé, dm-drogerie market.The location is absolutely quiet. Bed linen, towels говорим по русски!'

# Detect language and sort by it

In [18]:
# detect language
with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        try: 
            if listings.loc[id, "lang"].isnull().values.any():
                language = translate.detect(list(text))
                listings.loc[id, "lang"] = language
        except:
            continue
        last_id = id[-1]
        print(i)

0
1
2
3
4


In [19]:
listings = listings.sort_values(by="lang")

In [20]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"
listings.to_pickle(path)

In [21]:
listings["lang"].value_counts()

en    511
de    478
es      2
ru      2
hu      1
it      1
ja      1
nl      1
tr      1
zh      1
Name: lang, dtype: int64

# Translate text

In [22]:
#@title this is script.py
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=50)

path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"
listings = pd.read_pickle(path)

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

dataset = Translatedata(listings, "description")
dataloader = DataLoader(dataset, batch_size=50)


with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if listings.loc[id, dataset.column +"_en"].isnull().values.any():

            sourcefinder = listings.loc[id, "lang"].unique()
            if len(sourcefinder) > 1:
                  print(sourcefinder)
                  sourcefinder = None
            else:
                print(i)
                sourcefinder = sourcefinder[0]

            translation = translate(list(text), "en", source = sourcefinder)
            listings.loc[id, dataset.column +"_en"] = translation
        else:
          print("jumped", i)
        
        listings.to_pickle(path)

0
1
2
3
4
5
6
7
8
['de' 'en']
10
11
12
13
14
15
16
17
18
['en' 'es' 'hu' 'it' 'ja' 'nl' 'ru' 'tr' 'zh']


Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

In [23]:
listings = pd.read_pickle(path)

In [27]:
listings["description_en"].isna().any()

False

In [28]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings_secure_save.pickle"
listings.to_pickle(path)

In [ ]:
#@title new script
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=64)

path = "/content/drive/MyDrive/Colab/airbnb/data/translations/host_about"
listings = pd.read_pickle(path)

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

dataset = Translatedata(listings, "host_about")
dataloader = DataLoader(dataset, batch_size=64)

jump_value=0
exit_value=1000

with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if i < jump_value:
            print("jumped", i)
            continue
        if i > exit_value:
            break
        if listings.loc[id, dataset.column +"_en"].isnull().values.any():
            print(i)
            translation = translate(list(text), "en")
            listings.loc[id, dataset.column +"_en"] = translation
        else:
            print("already done in ", i)

        listings.to_pickle(path)

In [35]:
listings["description_en"].iloc[0]

'This is a 3 room apartment, kitchen with washing machine, coffee maker, kettle, toster and everything that can be seen in the pictures, Wi-Fi and TV in each room. Up to the underground Forstenrieder avenue are 4 min. walk, then 10 min. to Marienplatz. Next to the underground station are REWE, bowling, ca-fé, dm-drogerie market.The location is absolutely quiet. Bed linen, towels говорим по русски!'